In [13]:
from mlflow.tracking import MlflowClient
MLFLOW_TRACKING_URI="sqlite:///mlflow.db"
client=MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [14]:
client.search_experiments()

[<Experiment: artifact_location='/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/test/mlruns/1', creation_time=1716583599823, experiment_id='1', last_update_time=1716583599823, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716583182707, experiment_id='0', last_update_time=1716583182707, lifecycle_stage='active', name='Default', tags={}>]

In [15]:
client.create_experiment(name="my-cool-experiment")

'2'

In [28]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',    # Experiment ID we want
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [29]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 3097d21527074e1e97b345d49a5a8f69, rmse: 6.3054
run id: 4d43d889a57a4c7d915925ba03122fdf, rmse: 6.3154
run id: 958d6cd7394640739c766f2fb5296879, rmse: 6.3154
run id: 6f06cc97af3d4c1cbd2b7dd55983f077, rmse: 6.3154
run id: 6efcd821c88841bcbbaa841b21e25369, rmse: 6.3154


In [30]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

run_id = "6f06cc97af3d4c1cbd2b7dd55983f077"
model_uri = f"runs:/{run_id}/models"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1716618090524, current_stage='None', description=None, last_updated_timestamp=1716618090524, name='nyc-taxi-regressor', run_id='6f06cc97af3d4c1cbd2b7dd55983f077', run_link=None, source='/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/test/mlruns/1/6f06cc97af3d4c1cbd2b7dd55983f077/artifacts/models', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [31]:
model_uri

'runs:/6f06cc97af3d4c1cbd2b7dd55983f077/models'

In [32]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: Staging


/tmp/ipykernel_138219/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [37]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_138219/1957332551.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716618090524, current_stage='Staging', description='The model version 3 was transitioned to Production on 2024-05-25', last_updated_timestamp=1716618800765, name='nyc-taxi-regressor', run_id='6f06cc97af3d4c1cbd2b7dd55983f077', run_link=None, source='/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/test/mlruns/1/6f06cc97af3d4c1cbd2b7dd55983f077/artifacts/models', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [38]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1716618090524, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2024-05-25', last_updated_timestamp=1716618804161, name='nyc-taxi-regressor', run_id='6f06cc97af3d4c1cbd2b7dd55983f077', run_link=None, source='/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/test/mlruns/1/6f06cc97af3d4c1cbd2b7dd55983f077/artifacts/models', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [47]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [46]:
!ls

data			   mlflow.db  models	 requirements.txt
duration-prediction.ipynb  mlruns     new.ipynb


In [48]:
df = read_dataframe("data/yellow_tripdata_2023-01.parquet")


In [50]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

MlflowException: The following failures occurred while downloading one or more artifacts from /home/ubuntu/mlops-zoomcamp/02-experiment-tracking/test/mlruns/1/6f06cc97af3d4c1cbd2b7dd55983f077/artifacts:
##### File preprocessor #####
[Errno 2] No such file or directory: '/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/test/mlruns/1/6f06cc97af3d4c1cbd2b7dd55983f077/artifacts/preprocessor'